In [6]:
import pandas as pd

file_path = 'test_down_sampled.csv'
data = pd.read_csv(file_path)


data_cleaned = data.dropna(subset=['label'])
example_news_texts = data_cleaned['text'].tolist()
example_actual_labels = data_cleaned['label'].astype(int).tolist()
data_cleaned.head()


,text,label,sentiment
0,克林頓在1999年中國使館被炸後令人震撼的秘密講話！\n\n環球觀察者 CCCPPR-Can...,1.0,[0.49233323 0.25445962 0.25320718]
1,動員五百人 小英養二十五個黑機關\n身兼中華職棒聯盟會長的藍委吳志揚也認為，新政府宣示的發展...,1.0,[0.10994015 0.22028746 0.66977233]
2,趙曉慧：【#美國新任總統，#根本尚未出爐】\n\n台灣媒體幾乎一面倒，跟風美國左派媒體的假新...,1.0,[0.15548167 0.16697745 0.6775409 ]
3,(轉貼)轉載紀錄📝\n\n6/6高雄霸凌日，我不談投票，只想講一個真實故事：\n .\n 認...,1.0,[0.21979059 0.35442767 0.4257817 ]
4,省錢省電都是騙人。如果台灣核三核四的正常供電。電費可以充足有餘。全民的電費會更低更省錢。\n...,1.0,[0.31936854 0.22099052 0.45964095]


In [2]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00


In [9]:
import requests
import time
import openai
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from typing import List

In [4]:
def query_gpt4(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-0125-preview",
            messages=[{"role": "system", "content": "You are a helpful fake news detection assistant."}, # a context-setting mechanism for GPT-4
                      {"role": "user", "content": prompt}]
        )
        return response.choices[0].message['content']
    except Exception as e:
        print("Error querying OpenAI API:", e)
        return None

In [7]:
openai.api_key = "my_openai_api_key"

In [11]:
def gpt4_prompt_fake_news_detection(news_text: str) -> int:
    """
    Constructs a prompt for GPT-4 to classify the news as fake or true and uses GPT-4 response.

    Args:
        news_text (str): The news text to classify.

    Returns:
        int: 1 if the news is predicted to be fake, 0 if the news is predicted to be true.
    """

    prompt = (
        f"請根據以下新聞內容判斷新聞的真偽，並僅以數字0與1作答。回答1代表「假新聞」，0代表「真新聞」。新聞內容：\"{news_text}\""
    )

    response_content = query_gpt4(prompt)

    try:
        prediction = int(response_content.strip())
    except ValueError:
        print("Received an invalid response from GPT-4:", response_content)
        prediction = None

    return prediction

**Test**

In [8]:
first_news_text = data_cleaned['text'].iloc[0]

prompt = (
    f"請根據以下新聞內容判斷新聞的真偽，並僅以數字0與1作答。回答1代表「假新聞」，0代表「真新聞」。新聞內容：\"{first_news_text}\""
)

response_content = query_gpt4(prompt)

try:
    prediction = int(response_content.strip())
    print(f"Prompt:\n{prompt}\n")
    print(f"GPT-4's Response: {response_content}")
    print(f"Interpreted Prediction: {'Fake News (1)' if prediction == 1 else 'True News (0)'}")
except ValueError:
    print("Received an invalid response from GPT-4:", response_content)

Prompt:
請根據以下新聞內容判斷新聞的真偽，並僅以數字0與1作答。回答1代表「假新聞」，0代表「真新聞」。新聞內容："克林頓在1999年中國使館被炸後令人震撼的秘密講話！

環球觀察者 CCCPPR-Canada-China 2019-06-30 
克林頓在1999年中國使館被炸後的秘密講話！震撼囯人！

克林頓在99年中國使館被炸後的秘密講話曝光！

1999年5月13日，在5枚導彈轟炸中國駐南斯拉夫大使館後，由於美國國會的議員們強烈要求說明真相，克林頓總統來到國會發表了秘密講話。

「首先，我必須申明，我的以下講話，將涉及我們國家的最高利益，當然也是最高機密。雖然在這個星球上，我們不用怕誰，但是，在當前這個階段我們也不想製造任何不必要的麻煩。」

「眾所周知，冷戰結束以來，就美國的綜合國力而言，我們在全球不僅沒有對手，而且完全有能力控制全球。」

「我所要描述的新戰略的目標將是從現在起，美國將成為人類的最後唯一的帝國。」

「我們制定了詳細的目標：第一步：北約東擴。諸位已經看到，在科索沃戰事上，原來我們還對他(俄羅斯)心有餘悸，現在，他幾乎成了一絲不掛的乞丐。

但是，我想諸位議員能夠明確地瞭解，我們的東擴，不僅僅是讓俄羅斯出出洋相，我們東擴的目的是控制整個歐亞板塊，在那裡，在東南亞有一個我們頭痛的地方，就是中國。」

「總之，在這條路上雖然也有一點冒險性，但基本上沒有可以阻擋我們的敵人。一個完全在美國領導下的21世紀很快就會到來。」(全場掌聲)

接下來，克林頓針對議員們的提問，將美國的戰略意圖表露得清清楚楚。

一議員：總統閣下，科索沃戰事久拖不決，費用一增再增，到底還要打多久？

克林頓：縱觀一部美國的發展和發達的歷史，二次世界大戰給我國經濟帶來高速發展已經足以證明，戰爭符合美國的利益。

一議員：科索沃是否可能成為下一個朝鮮或越南，讓我們陷入戰爭的泥潭？

克林頓：海灣戰爭已證明瞭我們在戰爭中的傷亡幾乎等於零，這就像玩一個電子遊戲。所不同的是，他們是在用他們的生命和不多的、必須賴以生存的家當與我們賭博。

一議員：在科索沃戰爭中我們的一再失誤，造成1500多名平民傷亡和中國駐南大使館被炸，這是否證明，我們的戰爭機器已不再受控制或有別的原因？

克林頓：我想在這裡強調的是，所有的轟炸目標都是既定目標。同時，我想引用中國領導人的一句話：炸完軍事目標、道

**Prompting Task**

In [13]:
def predict_and_evaluate(news_texts: List[str], actual_labels: List[int]) -> float:
    """
    Predicts labels for a list of news texts and evaluates the accuracy directly by comparing
    each prediction with the actual label.

    Args:
        news_texts (List[str]): A list of news texts to classify.
        actual_labels (List[int]): The actual labels corresponding to the news texts.

    Returns:
        float: The accuracy of the predictions.
    """
    correct_predictions = 0
    valid_predictions = 0

    for text, actual_label in tqdm(zip(news_texts, actual_labels), desc="Processing news", total=len(news_texts)):
        prediction = gpt4_prompt_fake_news_detection(text)
        if prediction is not None:  # Ensure prediction is valid
            valid_predictions += 1
            if prediction == actual_label:
                correct_predictions += 1

    if valid_predictions > 0:
        accuracy = correct_predictions / valid_predictions
    else:
        accuracy = 0

    return accuracy

example_news_texts = data_cleaned['text'].tolist()
example_actual_labels = data_cleaned['label'].astype(int).tolist()

accuracy = predict_and_evaluate(example_news_texts, example_actual_labels)

print(f'Accuracy: {accuracy * 100:.2f}%')

Processing news:  64%|██████▍   | 136/212 [03:23<04:52,  3.85s/it]

Received an invalid response from GPT-4: 對不起，我無法直接提供新聞或事件的真實性評估，因為我無法存取當前的數據或最新消息。我建議你查找可靠的新聞來源或官方發布的信息以確認該新聞的真實性。


Processing news:  66%|██████▌   | 140/212 [03:35<04:52,  4.06s/it]

Received an invalid response from GPT-4: 抱歉，我無法直接判斷新聞真偽僅憑提供的信息。欲了解新聞真偽，建議透過官方或可信來源進行查證。


Processing news:  72%|███████▏  | 153/212 [03:56<03:32,  3.60s/it]

...

Received an invalid response from GPT-4: 抱歉，我無法直接回答新聞真偽的問題。


Processing news:  80%|████████  | 170/212 [04:28<02:44,  3.90s/it]

Received an invalid response from GPT-4: 抱歉，但我無法直接存取當前或最近的新聞來源進行即時確認。因此，我無法提供針對特定新聞標題或內容真偽的直接評估。建議查閱可靠新聞來源或官方聲明以獲得準確信息。


Processing news: 100%|██████████| 212/212 [05:08<00:00,  1.46s/it]

Accuracy: 65.70%


**LIAR Dataset**

In [15]:
liar_dataset_path = 'test.csv'
column_names = [
    'id', 'label', 'statement', 'subjects', 'speaker', 'speaker_title',
    'state', 'party', 'not_true_counts', 'barely_true_counts',
    'half_true_counts', 'mostly_true_counts', 'pants_fire_counts', 'context'
]

liar_data = pd.read_csv(liar_dataset_path, names=column_names, header=None)

print(liar_data[['statement', 'label']].head())

                                           statement       label
0  Building a wall on the U.S.-Mexico border will...        true
1  Wisconsin is on pace to double the number of l...       false
2  Says John McCain has done nothing to help the ...       false
3  Suzanne Bonamici supports a plan that will cut...   half-true
4  When asked by a reporter whether hes at the ce...  pants-fire


In [22]:
def liar_prompt_gpt4(content_text: str) -> str:
    """
    Constructs a prompt for GPT-4 to classify the content into one of six labels.

    Args:
        content_text (str): The content text to classify.

    Returns:
        str: The label predicted by GPT-4, or 'invalid' if the response is not recognized.
    """
    prompt = (
        f"Please classify the following content into one of these six categories (and answer in only one of the six labels): "
        f"true, mostly-true, half-true, barely-true, false, pants-on-fire. "
        f"Content: \"{content_text}\""
    )

    response_content = query_gpt4(prompt)

    if response_content is None:
        print(f"Failed to receive a valid response for the content: \"{content_text}\"")
        return 'invalid'

    label = response_content.strip().lower()

    # List of valid labels
    valid_labels = ["true", "mostly-true", "half-true", "barely-true", "false", "pants-on-fire"]

    if label not in valid_labels:
        print(f"Received an invalid response from GPT-4: '{response_content}'")
        return 'invalid'

    return label


def predict_labels(contents: List[str]) -> List[str]:
    """
    Predicts labels for a list of content texts.

    Args:
        contents (List[str]): A list of content texts to classify.

    Returns:
        List[str]: Predicted labels by GPT-4.
    """
    predictions = [liar_prompt_gpt4(content) for content in contents]
    return predictions

def calculate_accuracy(predicted_labels: List[str], actual_labels: List[str]) -> float:
    """
    Calculates the accuracy of predictions against actual labels for multi-class classification,
    excluding 'invalid' responses.

    Args:
        predicted_labels (List[str]): Predicted labels.
        actual_labels (List[str]): Actual labels.

    Returns:
        float: The accuracy score.
    """
    valid_pairs = [(p, a) for p, a in zip(predicted_labels, actual_labels) if p != 'invalid']
    if not valid_pairs:
        return 0

    correct_predictions = sum(p == a for p, a in valid_pairs)
    accuracy = correct_predictions / len(valid_pairs)
    return accuracy

liar_data_sample = liar_data[['statement']].head(200)
liar_labels_sample = liar_data[['label']].head(200)

predicted_labels = predict_labels(tqdm(liar_data_sample['statement'].tolist(), desc="Classifying content"))
accuracy = calculate_accuracy(predicted_labels, liar_labels_sample['label'].tolist())

print(f'Accuracy: {accuracy * 100:.2f}%')

Classifying content:  24%|██▍       | 49/200 [00:43<02:21,  1.07it/s]

Received an invalid response from GPT-4: 'mostly-false'


Classifying content:  61%|██████    | 122/200 [01:47<00:54,  1.42it/s]

Received an invalid response from GPT-4: 'mostly-false'


Classifying content: 100%|██████████| 200/200 [02:51<00:00,  1.16it/s]

Accuracy: 21.21%
